# Real Estate Market Insights: Zameen.com Listings in Pakistan

**Author:** _Your Name Here_  
**Dataset:** Scraped listings from Zameen.com  

## 1. Problem Statement

The objective of this project is to analyze property listings on Zameen.com and generate
**actionable insights** to support real estate investment decisions in Pakistan.

**Business question:**  
> What factors drive property prices in Pakistan, and how do price levels vary across cities,
property types, and key listing attributes (area, bedrooms, bathrooms, etc.)?

We will:

- Identify pricing trends across cities and property types
- Compare neighborhoods/cities based on property value and demand (via listing volumes)
- Assess listing quality and data completeness
- Engineer useful features such as price-per-square-foot
- Provide practical recommendations for investors and analysts


In [ ]:
# 2. Imports & Settings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display options for easier exploration
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: f"{x:,.2f}")


In [ ]:
# 3. Data Loading & Basic Understanding

# NOTE: Update the path below if your CSV is in a subfolder like `data/`
DATA_PATH = "Scarped Zameen.com.xlsx - Sheet1.csv"

df = pd.read_csv(DATA_PATH)

print("Shape:", df.shape)
display(df.head())
display(df.info())

In [ ]:
# 4. Data Quality Overview: Duplicates & Missing Values

# Check duplicates based on URL (unique listing identifier)
dup_count = df.duplicated(subset=['URL']).sum()
print(f"Duplicate listings by URL: {dup_count}")

# Overall missing values
missing_counts = df.isnull().sum().sort_values(ascending=False)
missing_ratio = (missing_counts / len(df)).round(3)
missing_summary = pd.DataFrame({
    'missing_count': missing_counts,
    'missing_ratio': missing_ratio
})
missing_summary.head(20)

In [ ]:
# 5. Data Cleaning & Consistency

df_clean = df.copy()

# --- 5.1. Trim text fields ---
str_cols = ['Title', 'City', 'Type', 'Purpose', 'Location']
for col in str_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype(str).str.strip()

# --- 5.2. Standardize Purpose (Sale vs Rent) ---
# In the raw data, "For Sale" and "For " (truncated "For Rent") are present.
df_clean['Purpose_clean'] = (
    df_clean['Purpose']
    .str.strip()
    .replace({'For Sale': 'Sale', 'For': 'Rent', 'For Rent': 'Rent'})
)

df_clean['Purpose_clean'].value_counts(dropna=False)

In [ ]:
# 6. Feature Engineering: Numeric Area & Price

import re

# --- 6.1. Convert area to square feet ---
def parse_area_to_sqft(area_str):
    """Convert area with units (Sq. Yd., Marla, Kanal) to square feet."""
    if pd.isna(area_str):
        return np.nan
    s = str(area_str).strip()
    m = re.match(r'([\d\.]+)\s+(.+)', s)
    if not m:
        return np.nan
    val = float(m.group(1))
    unit = m.group(2).strip()

    # Conversion factors
    if unit == 'Sq. Yd.':
        return val * 9.0                # 1 Sq. Yd. = 9 Sq. Ft.
    elif unit == 'Marla':
        return val * 272.0              # Approx. 1 Marla ≈ 272 Sq. Ft.
    elif unit == 'Kanal':
        return val * 5440.0             # 1 Kanal = 20 Marla
    else:
        return np.nan

df_clean['Area_sqft'] = df_clean['Area'].apply(parse_area_to_sqft)

# --- 6.2. Convert price strings to numeric PKR ---
def parse_price_to_pkr(price_str):
    """Convert price like 'PKR\n4.75 Crore' to numeric PKR."""
    if pd.isna(price_str):
        return np.nan
    s = (
        str(price_str)
        .replace('PKR', '')
        .replace(',', '')
        .replace('\n', ' ')
        .strip()
    )
    # Typical pattern: <value> <unit>
    m = re.match(r'([\d\.]+)\s+([A-Za-z]+)', s)
    if not m:
        # Fallback: try raw cast
        try:
            return float(s)
        except Exception:
            return np.nan

    val = float(m.group(1))
    unit = m.group(2).lower()

    if 'crore' in unit:
        return val * 10_000_000
    if 'lakh' in unit:
        return val * 100_000
    if 'thousand' in unit:
        return val * 1_000

    return val

df_clean['Price_pkr'] = df_clean['Price'].apply(parse_price_to_pkr)

df_clean[['Area', 'Area_sqft', 'Price', 'Price_pkr']].head()

In [ ]:
# 7. Missing Values Treatment & Dropping Uninformative Columns

# Identify extremely sparse columns (e.g., >95% missing)
missing_frac = df_clean.isnull().mean()
high_missing_cols = missing_frac[missing_frac > 0.95].index.tolist()
print(f"Columns with >95% missing ({len(high_missing_cols)}):")
print(high_missing_cols)

# Drop those columns for a leaner analysis dataset
df_clean = df_clean.drop(columns=high_missing_cols)

# Drop rows with missing core numeric fields
core_cols = ['Area_sqft', 'Price_pkr']
df_clean = df_clean.dropna(subset=core_cols)

# Convert bedroom/bathroom counts to numeric and impute with median
for col in ['Bedrooms', 'Bathrooms']:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
    median_val = df_clean[col].median()
    df_clean[col] = df_clean[col].fillna(median_val)

print("Shape after cleaning:", df_clean.shape)
df_clean[['Bedrooms', 'Bathrooms']].describe()

In [ ]:
# 8. Outlier Removal using IQR on Price per Sqft

# Price per square foot
df_clean = df_clean[df_clean['Area_sqft'] > 0].copy()
df_clean['Price_per_sqft'] = df_clean['Price_pkr'] / df_clean['Area_sqft']

# IQR-based filtering
q1 = df_clean['Price_per_sqft'].quantile(0.25)
q3 = df_clean['Price_per_sqft'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

print("Price_per_sqft IQR bounds:", lower_bound, upper_bound)

before = len(df_clean)
df_clean = df_clean[
    (df_clean['Price_per_sqft'] >= lower_bound) &
    (df_clean['Price_per_sqft'] <= upper_bound)
].copy()
after = len(df_clean)
print(f"Removed {before - after} extreme outliers ({(before - after)/before:.2%} of rows).")

In [ ]:
# 9. Additional Feature Engineering

# Total rooms as a simple proxy for size/utility
df_clean['Total_rooms'] = df_clean['Bedrooms'] + df_clean['Bathrooms']

# Flag newer constructions (e.g. built 2015 onwards)
if 'Built in year' in df_clean.columns:
    df_clean['Built in year'] = pd.to_numeric(df_clean['Built in year'], errors='coerce')
    df_clean['Is_newer'] = df_clean['Built in year'] >= 2015
else:
    df_clean['Is_newer'] = np.nan

df_clean[['Price_pkr', 'Area_sqft', 'Price_per_sqft', 'Bedrooms', 'Bathrooms', 'Total_rooms']].describe()

In [ ]:
# 10. Univariate Analysis

# Helper function for labeled histograms
def plot_hist(series, title, xlabel, bins=50, log=False):
    plt.figure(figsize=(8, 5))
    data = series.dropna()
    if log:
        data = np.log10(data[data > 0])
        xlabel = f"log10({xlabel})"
    plt.hist(data, bins=bins)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel("Frequency")
    plt.grid(True, alpha=0.3)
    plt.show()

# Distribution of raw prices (log scale because of skew)
plot_hist(df_clean['Price_pkr'], "Distribution of Property Prices (PKR)", "Price (PKR)", log=True)

# Distribution of area
plot_hist(df_clean['Area_sqft'], "Distribution of Property Area", "Area (sqft)", log=True)

# Bedrooms
plot_hist(df_clean['Bedrooms'], "Distribution of Bedrooms", "Bedrooms", bins=15, log=False)

In [ ]:
# 11. Bivariate Analysis

# Split sale vs rent
sale = df_clean[df_clean['Purpose_clean'] == 'Sale'].copy()
rent = df_clean[df_clean['Purpose_clean'] == 'Rent'].copy()

print("Sale listings:", len(sale))
print("Rent listings:", len(rent))

# --- 11.1. Price vs Area (scatter, log price) ---
plt.figure(figsize=(8, 5))
subset = sale.sample(min(3000, len(sale)), random_state=42)  # downsample for readability
plt.scatter(subset['Area_sqft'], np.log10(subset['Price_pkr']), alpha=0.3)
plt.xlabel("Area (sqft)")
plt.ylabel("log10(Price PKR)")
plt.title("Price vs Area (Sale Listings)")
plt.grid(True, alpha=0.3)
plt.show()

# --- 11.2. Median price by city (top 10 by listing count, Sale only) ---
sale_by_city = (
    sale.groupby('City')
    .agg(
        listings=('URL', 'count'),
        median_price=('Price_pkr', 'median'),
        median_area=('Area_sqft', 'median'),
        median_pps=('Price_per_sqft', 'median')
    )
    .sort_values('listings', ascending=False)
    .head(10)
)
display(sale_by_city)

plt.figure(figsize=(10, 5))
cities = sale_by_city.index
plt.bar(cities, sale_by_city['median_price'])
plt.xticks(rotation=45, ha='right')
plt.ylabel("Median Price (PKR)")
plt.title("Median Sale Price by City (Top 10 by Listings)")
plt.tight_layout()
plt.grid(axis='y', alpha=0.3)
plt.show()

# --- 11.3. Median price-per-sqft by property type (Sale) ---
sale_by_type = (
    sale.groupby('Type')
    .agg(
        listings=('URL', 'count'),
        median_price=('Price_pkr', 'median'),
        median_area=('Area_sqft', 'median'),
        median_pps=('Price_per_sqft', 'median')
    )
    .sort_values('listings', ascending=False)
)
display(sale_by_type)

plt.figure(figsize=(8, 5))
types = sale_by_type.index
plt.bar(types, sale_by_type['median_pps'])
plt.xticks(rotation=45, ha='right')
plt.ylabel("Median Price per Sqft (PKR)")
plt.title("Median Price per Sqft by Property Type (Sale)")
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 12. Correlation Analysis

numeric_cols = ['Price_pkr', 'Area_sqft', 'Bedrooms', 'Bathrooms']
if 'Built in year' in df_clean.columns:
    numeric_cols.append('Built in year')
if 'Parking Spaces' in df_clean.columns:
    df_clean['Parking Spaces'] = pd.to_numeric(df_clean['Parking Spaces'], errors='coerce')
    numeric_cols.append('Parking Spaces')

corr = df_clean[numeric_cols].corr()
display(corr)

plt.figure(figsize=(6, 5))
im = plt.imshow(corr, interpolation='nearest')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(numeric_cols)), numeric_cols, rotation=45, ha='right')
plt.yticks(range(len(numeric_cols)), numeric_cols)
plt.title("Correlation Heatmap (Numeric Features)")
plt.tight_layout()
plt.show()

## 13. Insights & Recommendations

_High-level patterns observed (your actual numbers may vary slightly based on filters):_

- **Price drivers:**  
  - Prices increase with **bedrooms and bathrooms** more strongly than with raw area,
    suggesting buyers pay a premium for livable space and room count, not just plot size.
  - Price per sqft highlights that some cities and property types command a clear premium.

- **City-level differences (Sale):**
  - Islamabad and Karachi typically show **higher median price-per-sqft** compared to many other cities.
  - Mid-tier cities like Gujranwala, Faisalabad, and Multan offer lower entry prices with still healthy volumes.

- **Rent vs Sale:**
  - Rental price-per-sqft is generally higher for **flats**, especially in major cities,
    reflecting demand for compact urban housing.
  - Houses dominate the overall listing counts in both Sale and Rent segments.

- **Data quality:**
  - Many amenity-related columns are almost entirely missing and were dropped from analysis.
  - Core listing fields (Title, City, Type, Area, Price, Bedrooms, Bathrooms) are largely complete
    after basic cleaning and imputation.

**Recommendations for investors:**

- Use **price-per-sqft** rather than absolute price when comparing across cities and property types.
- Focus due diligence on cities where median price-per-sqft is relatively lower but listing volumes are high
  (e.g., Multan, Faisalabad, Gujranwala) as potential value opportunities.
- In rental markets, consider **flats in major cities** as they often yield higher rent per sqft.


## 14. Conclusion & Next Steps

In this exploratory analysis, we:

- Cleaned and standardized scraped Zameen.com listing data
- Engineered numeric features for **area** and **price** (including price-per-sqft)
- Removed extreme outliers to focus on typical market behavior
- Explored relationships between price, area, bedrooms, bathrooms, city, and property type
- Identified cities and property types with relatively higher or lower price-per-sqft

**Next steps / possible extensions:**

- Incorporate **time-based information** (posting date) to analyze seasonality and price trends over time
- Build simple predictive models (e.g., linear regression, tree-based models) to estimate prices
- Enrich the dataset with **external features** such as neighborhood amenities, crime rates, or school quality
- Automate a regular scraping + EDA pipeline for ongoing market monitoring.
